# Replication - Standard Case 1 - Table

Here we provide a notebook to replicate the summary tables for the standard case 1 simulation. 

The notebook replicates the results in:
- /out/simulation/tables/sim_sc1*

The main script can be found at: 
- /scripts/simulation/tables/standard_case1.py


## Please choose the settup for replication:

In [ ]:
batchsize = '512' # 64, 128, 256, 512, full

In [ ]:
# google colab specific - installing probcox
!pip3 install probcox

In [ ]:
# Modules
# =======================================================================================================================
import os
import sys

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

sim_name = 'sim_sc1'

In [ ]:
# Simulation Settings
# =======================================================================================================================

I = 1000
P = 6
url = 'https://raw.githubusercontent.com/alexwjung/ProbCox/main/paper/ProbCox/out/simulation/sim_sc1/theta.txt?token=AIVIKKF7OPFZDYEFVQC3U5TATO3Y6'
theta = np.asarray(pd.read_csv(url, header=None, sep=';'))


In [ ]:
# Overall Parameters
# =======================================================================================================================


url = 'https://raw.githubusercontent.com/alexwjung/ProbCox/main/paper/ProbCox/out/simulation/sim_sc1/N_obs.txt?token=AIVIKKBXQZSDGMN3M3UWEC3ATO3Y2'
N_obs = pd.read_csv(url, header=None, sep=';')


print('Mean number of intervall observations: ', np.mean(N_obs.iloc[:, 1]))
print('Mean number of censorship: ', 1 - (np.mean(N_obs.iloc[:, 2])/I))

Mean number of intervall observations:  7629.5
Mean number of censorship:  0.782425


In [ ]:
# empty file to write results into
res = np.zeros((P, 6))
res[:, 0] = theta[:, 0]

url1 = 'https://raw.githubusercontent.com/alexwjung/ProbCox/main/paper/ProbCox/out/simulation/sim_sc1/probcox' + str(batchsize) +'_theta.txt?token=AIVIKKFQBYRXV6PNA2ZAM5LATKYFQ'
url2 = 'https://raw.githubusercontent.com/alexwjung/ProbCox/main/paper/ProbCox/out/simulation/sim_sc1/probcox' + str(batchsize) +'_theta_lower.txt?token=AIVIKKHNLOUIFFTFHWMFN43ATKYFU'
url3 = 'https://raw.githubusercontent.com/alexwjung/ProbCox/main/paper/ProbCox/out/simulation/sim_sc1/probcox' + str(batchsize) +'_theta_upper.txt?token=AIVIKKA62RCC7HUDKLZNRO3ATKYFY'
theta_est = pd.read_csv(url1, header=None, sep=';')
theta_est_lower = pd.read_csv(url2, header=None, sep=';')
theta_est_upper = pd.read_csv(url3, header=None, sep=';')

theta_est = theta_est.dropna(axis=0)
theta_est = theta_est.groupby(0).first().reset_index()
theta_est = theta_est.iloc[:, :-1]
assert theta_est.shape[0] == 200

theta_est_lower = theta_est_lower.dropna(axis=0)
theta_est_lower = theta_est_lower.groupby(0).first().reset_index()
theta_est_lower = theta_est_lower.iloc[:, :-1]
assert theta_est_lower.shape[0] == 200

theta_est_upper = theta_est_upper.dropna(axis=0)
theta_est_upper = theta_est_upper.groupby(0).first().reset_index()
theta_est_upper = theta_est_upper.iloc[:, :-1]
assert theta_est_upper.shape[0] == 200

theta_bound = theta_est_lower.merge(theta_est_upper, how='inner', on=0)
theta_bound = theta_bound.merge(theta_est, how='inner', on=0)
theta_est = np.asarray(theta_bound.iloc[:, -P:]).astype(float)
theta_bound = theta_bound.iloc[:, :-P]
theta_bound = np.asarray(theta_bound.iloc[:, 1:]).astype(float)

res[:, 1] = np.mean(theta_est, axis=0)
res[:, 2] = np.sqrt(np.var(theta_est, axis=0))
res[:, 3] = np.sqrt(np.mean((theta_est - theta[:, 0][None, :])**2, axis=0))

ll = []
for ii in range(P):
    ll.append(np.mean(theta_bound[:, ii+P] - theta_bound[:, ii]))
res[:, 4] = np.asarray(ll)

ll = []
for ii in range(P):
    ll.append(np.sum(np.logical_and(theta[ii]>=theta_bound[:, ii], theta[ii]<=theta_bound[:, ii+P]))/theta_bound.shape[0])
res[:, 5] = np.asarray(ll)

res = np.round(res, 2)
pd.DataFrame(res)

# column headings
#$\theta$   $\bar{\hat{\theta}}$ 	$\overline{\sigma_{\hat{\theta}}}$	$RMSE$ 	$\overline{HPD}_{95\%}$	$Coverage_{95\%}$

,0,1,2,3,4,5
0,-0.9,-0.89,0.21,0.21,0.93,0.96
1,0.2,0.18,0.16,0.16,0.70,0.97
2,0.0,-0.04,0.18,0.18,0.74,0.96
3,-0.4,-0.39,0.07,0.07,0.30,0.96
4,1.1,1.09,0.08,0.08,0.32,0.95
5,0.0,0.01,0.07,0.07,0.30,0.96


In [ ]:
# R-Cox Table
# =======================================================================================================================

res = np.zeros((P, 6))
res[:, 0] = theta[:, 0]

url1 = 'https://raw.githubusercontent.com/alexwjung/ProbCox/main/paper/ProbCox/out/simulation/sim_sc1/R_theta.txt?token=AIVIKKHP5R2UZLFEJVHETDLATKHU4'
url2 = 'https://raw.githubusercontent.com/alexwjung/ProbCox/main/paper/ProbCox/out/simulation/sim_sc1/R_se.txt?token=AIVIKKCVNAYPRTZTNKN4INLATKHUY'

theta_est = pd.read_csv(url1, header=None, sep=';')
theta_se = pd.read_csv(url2, header=None, sep=';')

theta_est = theta_est.dropna(axis=0)
theta_est = theta_est.groupby(0).first().reset_index()
assert theta_est.shape[0] == 200

theta_se = theta_se.dropna(axis=0)
theta_se = theta_se.groupby(0).first().reset_index()
assert theta_se.shape[0] == 200

theta_mat = theta_est.merge(theta_se, how='inner', on=0)
theta_mat = np.asarray(theta_mat.iloc[:, 1:]).astype(float)

theta_est = theta_mat[:, :P]
theta_se = theta_mat[:, P:]

res[:, 1] = np.mean(theta_est, axis=0)
res[:, 2] = np.sqrt(np.var(theta_est, axis=0))
res[:, 3] = np.sqrt(np.mean((theta_est - theta[:, 0][None, :])**2, axis=0))

theta_est_lower = theta_est - 1.96*theta_se
theta_est_upper = theta_est + 1.96*theta_se

theta_bound = np.concatenate((theta_est_lower, theta_est_upper), axis=1)

ll = []
for ii in range(P):
    ll.append(np.mean(theta_bound[:, ii+P] - theta_bound[:, ii]))
res[:, 4] = np.asarray(ll)

ll = []
for ii in range(P):
    ll.append(np.sum(np.logical_and(theta[ii]>=theta_bound[:, ii], theta[ii]<=theta_bound[:, ii+P]))/theta_bound.shape[0])
res[:, 5] = np.asarray(ll)

res = np.round(res, 2)
pd.DataFrame(res)

# column headings
#$\theta$   $\bar{\hat{\theta}}$ 	$\overline{\sigma_{\hat{\theta}}}$	$RMSE$ 	$\overline{CI}_{95\%}$	$Coverage_{95\%}$

,0,1,2,3,4,5
0,-0.9,-0.92,0.23,0.23,0.91,0.94
1,0.2,0.20,0.16,0.16,0.64,0.97
2,0.0,-0.02,0.18,0.18,0.68,0.94
3,-0.4,-0.39,0.06,0.07,0.27,0.96
4,1.1,1.10,0.07,0.07,0.28,0.96
5,0.0,0.01,0.07,0.07,0.27,0.94
